<a href="https://colab.research.google.com/github/AbhiGupta1310/Multi-Task-NLP-Transformers/blob/main/Multi_Task_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline, logging
logging.set_verbosity_error()
import pandas as pd

In [2]:
# AI Assistant Class
class Assistant:

  # Object constructor
  def __init__(self):
    pass

  # Function for Sentiment Analysis: Understand the 'vibe' of a text
  def sentiment_analysis(self, review):
    sentiment_app = pipeline("sentiment-analysis",
                             model="distilbert-base-uncased-finetuned-sst-2-english")
    return sentiment_app(review)
  # Function for Summarization: Turn a long paragraph into a bite-sized nugget
  def summarization(self, long_text):
    summarizer = pipeline("summarization",
                          model= "sshleifer/distilbart-cnn-12-6")
    summary = summarizer(long_text, max_length=100, min_length=10)
    return summary[0]['summary_text']
  # Function for Named Entity Recognition (NER): Spot the 'Who', 'Where', and 'What'
  def ner(self, text):
    ner_tagger = pipeline("ner", aggregation_strategy="simple",
                          model="dbmdz/bert-large-cased-finetuned-conll03-english")
    entities = ner_tagger(text)
    entities_dict = [(entity['word'],entity['entity_group']) for entity in entities]
    return entities_dict
  # Function to add a topic to the summary
  def add_topic(self, text):
    topic_classifier = pipeline("zero-shot-classification",
                                # model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")
                                model= "facebook/bart-large-mnli")
    topic = topic_classifier(text,
                             candidate_labels=["compliment", "complaint", "suggestion", "statement"])
    return topic['labels'][0], topic['scores'][0]
  # Function to organize the results in a pandas dataframe
  def organize(self, texts):
    # Create an empty dataframe
    df = pd.DataFrame(columns=str.split('customer_name,sentiment,sentiment_conf,topic,topic_conf,summary',
                                        sep=","))
    # Loop through texts
    for text in texts:
      summary = ai.summarization(text)
      topic, topic_conf = ai.add_topic(text)
      sentiment = ai.sentiment_analysis(text)[0]['label']
      sentiment_conf = ai.sentiment_analysis(text)[0]['score']
      customer_name = ai.ner(text)[0][0]
    # Add text to the df
      df.loc[len(df)] = [customer_name, sentiment, sentiment_conf, topic, topic_conf, summary]
    return df

In [3]:
# SENTIMIENT

In [4]:
# Instantiate the class
ai = Assistant()

# Run sentiment analysis
review = "I tried the new software update. It's surprisingly intuitive and fast!"
print(f"Sentiment: {ai.sentiment_analysis(review)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Sentiment: [{'label': 'POSITIVE', 'score': 0.9995124340057373}]


In [5]:
# Summarization

In [6]:
# Run the Summarization
long_text = """
Hugging Face has democratized AI by providing a central hub for pre-trained models.
Before this, researchers had to spend weeks training models on expensive GPUs.
Now, with the 'transformers' library, a developer can implement SOTA (State-of-the-Art)
NLP tasks in just a few lines of code, significantly lowering the barrier to entry.
"""

print(f"Summary: {ai.summarization(long_text)}")

Summary:  Hugging Face has democratized AI by providing a central hub for pre-trained models . The 'transformers' library allows developers to implement SOTA (State-of-the-Art) NLP tasks in just a few lines of code .


In [7]:
# Name Recognition

In [8]:
# Run NER
business_news = "Apple is looking at buying a startup in London for $1 billion."
print(f"Named Entities: {ai.ner(business_news)}")

Named Entities: [('Apple', 'ORG'), ('London', 'LOC')]


In [9]:
# Zero-Shot Classification

In [10]:
# Run Topic Classifier (Zero-shot classification)
email = """Subject: New Feature Request
Hey, I love your app! One thing that would make it even better is if you could add a dark mode. It would be super helpful for my eyes at night. I've been using it a lot lately and it's really straining my eyes. I've tried changing my phone settings, but it would be great if the app had its own dark mode. That way, I could use it anytime without having to worry about the brightness. Keep up the good work, and I hope you consider my suggestion!
Best, Hanna Lotus."""

topic, score = ai.add_topic(email)
print(f"Topic: {topic}, Score: {score}")

Topic: suggestion, Score: 0.7827219367027283


In [11]:
# Emails received from customers
emails = [

"""Subject: Issue with Order #1234
Hi, I placed an order last week and it still hasn't arrived. Can you please look into this and let me know what's going on? I've tried tracking it but it says it's still in transit. I've checked my email and didn't see any updates from you guys, so I'm getting a bit worried. Could you please check on the status of my order and let me know when I can expect it to arrive?
My order number is #1234.
Regards, John Rick""",

"""Subject: New Feature Request
Hey, I love your app! One thing that would make it even better is if you could add a dark mode. It would be super helpful for my eyes at night. I've been using it a lot lately and it's really straining my eyes. I've tried changing my phone settings, but it would be great if the app had its own dark mode. That way, I could use it anytime without having to worry about the brightness. Keep up the good work, and I hope you consider my suggestion!
Best, Hanna Lotus.""",

"""Subject: Discount Request
Hi, I'm a big fan of your product and I was wondering if you'd be willing to offer a discount for bulk orders. I'm looking to purchase a few more items and I'd love to save some money. I've been recommending your product to all my friends and family, and I'm hoping you can give me a good deal. If not, no worries! I just thought I'd ask. But if you do offer discounts for bulk orders, I'd love to hear about it.
Thanks!
Carla Gibson""",
"""Subject: Great Service
Just wanted to say thanks for the excellent customer service I received yesterday. The representative was super helpful and resolved my issue quickly. I was really impressed with how efficiently they handled my problem, and they were very friendly and courteous. It's not often that you get good customer service these days, so I wanted to pass on my appreciation. Keep up the good work, and I'll definitely be recommending your company to others.
Regards, Jun-Li Chen""",
"""Subject: Problem with Payment
Hi, I'm having trouble completing my purchase. Every time I try to checkout, it says my payment method is declined. Can you please help me figure out what's going on? I've checked my account and I have enough funds, so I'm not sure what's going on. I've tried using a different card, but it's still not working. Can you please look into this and get back to me?
I'd really like to complete my purchase as soon as possible.
Best,
Rodrick Wings""",
"""Subject: Feature Suggestion
Hey, I've been using your product for a while now and I think it would be awesome if you could add a feature that lets us customize the dashboard. It would make it way more useful for me. I've been using it to track my tasks and projects, and it would be great if I could rearrange the layout to fit my needs. I've also noticed that some of my colleagues use different tools for similar tasks, and it would be great if your product could integrate with those tools. That way, we could use one platform for everything.
Just a thought!
Patrik Brown""",
"""Subject: Compliment
I just wanted to say thanks for making such a great product! It's really made a difference in my daily life and I'm so grateful. I've been using it for a few weeks now, and it's helped me streamline my workflow and stay organized.
The interface is so intuitive and easy to use, and the features are exactly what I need. I've already recommended it to my friends and family, and I'll definitely be recommending it to others.
Keep up the good work!
Best,
Julia Deeds""",
"""Subject: Discount Inquiry
Hi, I'm a student and I'm on a tight budget. Do you offer any discounts for students? I'd love to purchase your product, but it's a bit out of my price range right now. I've been looking for a tool like this to help me with my studies, and I think yours is the best one out there. If you do offer student discounts, I'd really appreciate it if you could let me know. If not, no worries!
I just thought I'd ask.
Paula Hom"""
]

In [15]:
# Run the assitant
df = ai.organize(emails)

# View DataFrame
df
